# **Iskander Ishkineev BS-AI-01**

# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import re
from collections import Counter
import pandas as pd
import numpy as np
import random
import string
import difflib
import time

In [2]:
count = 0
words = []
bigrt = {}
with open('bigrams.txt', 'r', encoding='latin-1') as file:
  lines = file.readlines()
  for i in lines:
    i = i.strip().split("\t")
    words.append(i[1])
    words.append(i[2])
    bigrt[f"{i[1]} {i[2]}"] = i[0]
    count += int(i[0])
WORDS = Counter(words)

In [3]:
for key in bigrt:
  bigrt[key] = int(bigrt[key])/count

In [4]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [5]:
def predict_first_word(word):
    "Predicts the most probable first word in a sentence. "
    possible_edits = edits1(word)
    possible_edits_2 = edits2(word)
    candidate_words = set(possible_edits).union(possible_edits_2)

    if word in WORDS:
        return word

    max_count = -1
    predicted_word = ''
    for candidate in candidate_words:
        if candidate in WORDS and WORDS[candidate] > max_count:
            max_count = WORDS[candidate]
            predicted_word = candidate

    return predicted_word

In [6]:
def predict_next_word(prev_word, current_word):
    "Predicts the most probable next word given the previous word and the current word."
    possible_edits = edits1(current_word)
    possible_edits_2 = edits2(current_word)
    candidate_words = set(possible_edits).union(possible_edits_2)

    pair = f"{prev_word} {current_word}"
    if pair in bigrt:
        return current_word

    max_probability = -1
    predicted_word = ''
    for candidate in candidate_words:
        candidate_pair = f"{prev_word} {candidate}"
        if candidate in WORDS:
            probability = bigrt.get(candidate_pair, 0)
            if probability > max_probability:
                max_probability = probability
                predicted_word = candidate

    if not predicted_word:
        for candidate in candidate_words:
            if candidate in WORDS:
                if WORDS[candidate] > max_probability:
                    max_probability = WORDS[candidate]
                    predicted_word = candidate

    return predicted_word


## Justify your decisions

`Bigram datasets was used`
since among given datasets, the one with bigrams was the largest, so there was larger variety of words for testing.

Also, for this task bigrams are more than sufficient to get the results since the context provided by bigrms is good enough.
Moreover bigrams provide high efficiency, compared to higher order grams that are computationally more expensive.


If a word or gram appears at the beginning or middle of an N-gram, its probability is determined by dividing 1 by the sum of all counts of bigrams (which can alternatively be expressed as the number of distinct known words). Conversely, if the word or gram is positioned at the end of an N-gram, its probability is calculated by dividing 1 by the sum of counts of known bigrams that can be formed from the preceding gram.

My solution utilizes a combination of edits on the input word, frequency counts of words, and probabilities from a bigram model to predict the most likely corrections or next words.








## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [7]:
def mutate_word(word):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'

    action = np.random.randint(0, 3)

    if action == 0:
        # replace a letter
        word = list(word)
        word[np.random.randint(0, len(word))] = alphabet[np.random.randint(0, 26)]
        word = ''.join(word)
    elif action == 1 and len(word) > 1:
        # remove a letter
        rem_index = np.random.randint(0, len(word))
        word = word[:rem_index] + word[rem_index+1:]
    else:
        # add a letter
        word = word + alphabet[np.random.randint(0, 26)]

    return word

In [60]:
simple_sentences = [
    "It was raining outside",
    "They tried to do the math test",
    "I am always annoyed by the loud noises",
    "They want to try this cake",
    "The ship cuts through the vast expanse of the ocean",
    "He works together with crew members",
    "The sun hangs high in the sky",
    "The smell of fresh bread fills the kitchen",
    "Snowflakes dance in the winter air",
    "Stars twinkle in the night sky"
]

split_sentences = []
for sentence in simple_sentences:
    split_sentences.append(sentence.lower().split())

edited_sentences = []
for sentence in split_sentences:
  mutated_sentence = ""
  for word in sentence:
    mutate = random.randint(0, 1)
    if mutate > 0:
      mutated_sentence += mutate_word(mutate_word(word)) + " "
    else:
      mutated_sentence += word + " "
  edited_sentences.append(mutated_sentence[:-1])

print(split_sentences)
print(edited_sentences)

[['it', 'was', 'raining', 'outside'], ['they', 'tried', 'to', 'do', 'the', 'math', 'test'], ['i', 'am', 'always', 'annoyed', 'by', 'the', 'loud', 'noises'], ['they', 'want', 'to', 'try', 'this', 'cake'], ['the', 'ship', 'cuts', 'through', 'the', 'vast', 'expanse', 'of', 'the', 'ocean'], ['he', 'works', 'together', 'with', 'crew', 'members'], ['the', 'sun', 'hangs', 'high', 'in', 'the', 'sky'], ['the', 'smell', 'of', 'fresh', 'bread', 'fills', 'the', 'kitchen'], ['snowflakes', 'dance', 'in', 'the', 'winter', 'air'], ['stars', 'twinkle', 'in', 'the', 'night', 'sky']]
['ity walg rainingtt outsidvl', 'wheyn trgedq jk doqp the math tesvu', 'i am always annoyed baf the loud noireu', 'thyyi wantxc to try this cake', 'the skipj utsz hroughm rh vamt epanse ofzj tter ocean', 'he works togeherl witn crew members', 'the sun hangs high ingx the sky', 'bhe smell pfi tesh bread filgso the kjtchn', 'snowfzakest dance in the winter aiq', 'stars twinkle ik te night sky']


In [9]:
def similarity(s1, s2):
    matcher = difflib.SequenceMatcher(None, s1.lower(), s2.lower())
    return matcher.ratio()

In [61]:
count = len(edited_sentences)
total = 0

for k in range(len(edited_sentences)):
  corrected = ''
  for i in range(len(edited_sentences[k].split())):
    start = time.time()
    if i == 0:
      corrected += predict_first_word(edited_sentences[k].split()[i]) + ' '
      current_word = predict_first_word(edited_sentences[k].split()[i])
    else:
      corrected += predict_next_word(current_word, edited_sentences[k].split()[i])+ ' '
      current_word = predict_next_word(current_word, edited_sentences[k].split()[i])
  corrected = corrected[:-1]
  print("Sentence before: ", edited_sentences[k])
  print("Sentence after: ", corrected)
  print("Correct sentence: ", simple_sentences[k])
  similarity_score = similarity(corrected, simple_sentences[k])
  print("Similarity score", similarity_score)
  total += similarity_score

overall_accuracy = total / count
print("Overall accuracy:", overall_accuracy)

Sentence before:  ity walg rainingtt outsidvl
Sentence after:  to talk raining outside
Correct sentence:  It was raining outside
Similarity score 0.8444444444444444
Sentence before:  wheyn trgedq jk doqp the math tesvu
Sentence after:  they tried to do the math test
Correct sentence:  They tried to do the math test
Similarity score 1.0
Sentence before:  i am always annoyed baf the loud noireu
Sentence after:  i am always enjoyed a the loud noise
Correct sentence:  I am always annoyed by the loud noises
Similarity score 0.8918918918918919
Sentence before:  thyyi wantxc to try this cake
Sentence after:  they want to try this cake
Correct sentence:  They want to try this cake
Similarity score 1.0
Sentence before:  the skipj utsz hroughm rh vamt epanse ofzj tter ocean
Sentence after:  the ship its rough on at ease of the ocean
Correct sentence:  The ship cuts through the vast expanse of the ocean
Similarity score 0.8387096774193549
Sentence before:  he works togeherl witn crew members
Sent

Norvig's solution

In [54]:
import re
from collections import Counter
import pandas as pd
import numpy as np

def words(text): return re.findall(r'\w+', text.lower())

file = open('big.txt').read()
WORDS = Counter(words(file))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]).union(known(edits1(word))) or known(edits2(word)) or [word])

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

In [59]:
count = len(edited_sentences)
total = 0

for k in range(len(edited_sentences)):
  corrected = ''
  for i in range(len(edited_sentences[k].split())):
    start = time.time()
    if i == 0:
      corrected += correction(edited_sentences[k].split()[i]) + ' '
      prev_word = correction(edited_sentences[k].split()[i])
    else:
      corrected += correction(edited_sentences[k].split()[i]) + ' '
      prev_word = correction(edited_sentences[k].split()[i])
  corrected = corrected[:-1]
  print("Sentence before: ", edited_sentences[k])
  print("Sentence after: ", corrected)
  print("Correct sentence: ", simple_sentences[k])
  similarity_score = similarity(corrected, simple_sentences[k])
  print("Similarity score", similarity_score)
  total += similarity_score

overall_accuracy = total / count
print("Overall accuracy:", overall_accuracy)

Sentence before:  tw was raining oxtsidea
Sentence after:  to was raising outside
Correct sentence:  It was raining outside
Similarity score 0.9090909090909091
Sentence before:  they tried d do khev math tekh
Sentence after:  the cried a to kiev path takh
Correct sentence:  They tried to do the math test
Similarity score 0.6440677966101694
Sentence before:  i am alys annoyedoc by the lou qoises
Sentence after:  in a alms annoyed by the you horses
Correct sentence:  I am always annoyed by the loud noises
Similarity score 0.821917808219178
Sentence before:  bey wacts g tryme this ake
Sentence after:  by acts a time his are
Correct sentence:  They want to try this cake
Similarity score 0.5416666666666666
Sentence before:  the shirl cuts through e vast expanse or e ocean
Sentence after:  the shirt cut through a last expense of a ocean
Correct sentence:  The ship cuts through the vast expanse of the ocean
Similarity score 0.8367346938775511
Sentence before:  hecn works tmgther wit crww debe

Based on the results above, we can see that:
* the Norvig's solution gives accuracy of ~ 80%,
* while my modified solution gives accuracy of ~ 90%.